# 뉴스그룹 분류 실습

In [1]:
# 데이터 로드
from sklearn.datasets import fetch_20newsgroups
# subset인자에 all로 train, test데이터 모두 가져오기
news_data = fetch_20newsgroups(subset='all', random_state=42)

In [2]:
# news_data는 dict형로되어 있음. 속성값들 확인해보기
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [4]:
# 클래스 값과 분포도 살펴보기
import pandas as pd
print('종속변수 클래스 값과 분포도\n',
     pd.Series(news_data.target).value_counts(ascending=False))
print('종속변수 클래스 이름들\n',
     news_data.target_names)

종속변수 클래스 값과 분포도
 10    999
15    997
8     996
9     994
11    991
13    990
7     990
5     988
14    987
2     985
12    984
3     982
6     975
1     973
4     963
17    940
16    910
0     799
18    775
19    628
dtype: int64
종속변수 클래스 이름들
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [6]:
# 데이터 하나만 살펴보기
print(news_data.data[0])

# 헤더값들에 텍스트의 주요 정보가 포함되어 있음

From: Mamatha Devineni Ratnam <mr47+@andrew.cmu.edu>
Subject: Pens fans reactions
Organization: Post Office, Carnegie Mellon, Pittsburgh, PA
Lines: 12
NNTP-Posting-Host: po4.andrew.cmu.edu



I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However, I am going to put an end
to non-PIttsburghers' relief with a bit of praise for the Pens. Man, they
are killing those Devils worse than I thought. Jagr just showed you why
he is much better than his regular season stats. He is also a lot
fo fun to watch in the playoffs. Bowman should let JAgr have a lot of
fun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final
regular season game.          PENS RULE!!!




In [10]:
# 데이터 분할
from sklearn.datasets import fetch_20newsgroups

train_data = fetch_20newsgroups(subset='train',
                               remove=('headers','footers','quotes'),
                               random_state=12)
X_train = train_data.data
y_train = train_data.target

test_data = fetch_20newsgroups(subset='test',
                              remove=('headers','footers','quotes'),
                              random_state=12)
X_test = test_data.data
y_test = test_data.target

print(f"학습 데이터 크기: {len(X_train)}\n 테스트 데이터 크기: {len(X_test)}")

학습 데이터 크기: 11314
 테스트 데이터 크기: 7532


In [11]:
# 텍스트 데이터 feature Vectorizer
from sklearn.feature_extraction.text import CountVectorizer

cnt_vect = CountVectorizer()
# CountVecorizer도 한번 fit 하면 검증, 테스트 데이터에도 똑같은 것으로 해야함!
# 다시 한 번 fit하게 되면 벡터화되는 feature 개수가 달라짐!
X_train_vec = cnt_vect.fit_transform(X_train)
X_test_vec = cnt_vect.transform(X_test)

# feature 벡터화시킨 후 shape 보기
print(f"학습 데이터 shape:{X_train_vec.shape}\n 테스트 데이터 shape:{X_test_vec.shape}")

학습 데이터 shape:(11314, 101631)
 테스트 데이터 shape:(7532, 101631)


In [12]:
# 로지스틱 리그레션으로 모델링하기
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr_clf = LogisticRegression()
lr_clf.fit(X_train_vec, y_train)
y_pred = lr_clf.predict(X_test_vec)
acc = accuracy_score(y_test, y_pred)
print(f"분류 정확도 : {acc : .4f}")

분류 정확도 :  0.6053


/Users/younghun/opt/anaconda3/envs/venvforpython/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [13]:
# Tf-idf로 feature vectorizer 시킨 후 로지스틱리그레션 해보기
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

tf_idf = TfidfVectorizer()
X_train_tf = tf_idf.fit_transform(X_train)
X_test_tf = tf_idf.transform(X_test)

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tf, y_train)
y_pred = lr_clf.predict(X_test_tf)
acc = accuracy_score(y_test, y_pred)
print(f"Tf-idf 벡터화 후 정확도 :{acc : .4f}")

Tf-idf 벡터화 후 정확도 : 0.6737


In [15]:
# Vecorizer 파라미터 추가한 후 적용해보기
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

tf_idf = TfidfVectorizer(ngram_range=(2,2), stop_words='english')
X_train_tf = tf_idf.fit_transform(X_train)
X_test_tf = tf_idf.transform(X_test)

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tf, y_train)
y_pred = lr_clf.predict(X_test_tf)
acc = accuracy_score(y_test, y_pred)
print(f"2gram, 불용어 제거 적용 후 정확도 :{acc :.4f}")
# ngram_range 2개이상으로 늘리면 성능 올라감

2gram, 불용어 제거 적용 후 정확도 :0.5150


In [16]:
# 위에서 Tf-idf 벡터화 시킨 후 GridSearchCV 사용해보기
from sklearn.model_selection import GridSearchCV

# 로지스틱 리그레션 규제 파라미터 C = 1/alpha임. 따라서 C값이 클수록 약한 규제!
C_params = {'C':[0.01, 0.1, 1]}
grid_cv_lr = GridSearchCV(lr_clf, param_grid=C_params,
                         cv=3, scoring='accuracy', verbose=1)
grid_cv_lr.fit(X_train_tf, y_train)
print("최적의 C 파리미터 값 :", grid_cv_lr.best_params_)

y_pred = grid_cv_lr.predict(X_test_tf)
acc = accuracy_score(y_test, y_pred)
print(f"최적의 Estimator로 학습시킨 후 정확도 :{acc:.4f}")

Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 10.7min finished


최적의 C 파리미터 값 : {'C': 1}
최적의 Estimator로 학습시킨 후 정확도 :0.5150


**``하단 2개의 모델은 시간이 오래걸려 동작 생략``**

In [ ]:
# feature 벡터화와 모델링 한번에 Pipeline으로 사용하기
from sklearn.pipeline import Pipeline
# Pipeline은 list안에 튜플('객체',과정) 식으로 선언
pipeline = Pipeline([('tf_idf', TfidfVectorizer(ngram_range=(2,2),
                                               stop_words='english'),
                     'lr_clf', LogisticRegression(C=1))])
# 파이프라인을 사용하면 벡토라이저 함수에서 fit_transform 없어도 자동으로 해줌
# 따라서 파이프라인의 fit인자에 벡터화시키기 전 train, test 데이터를 넣어주자
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"파이프라인을 적용한 정확도 : {acc : .4f}")

In [ ]:
# 이번엔 feature 벡터화와 GridSearchCV 한번에 파이프라인을 이용해서 수행하기
from sklearn.pipeline import Pipeline

pipeline = Pipeline([('tf_idf',TfidfVectorizer(stop_words='english')),
                     ('lr_clf',LogisitcRegression())])
# Pipeline 객체를 GridSearch 인자의 모델부분에 넣어주려면
# params 할당할 때 Pipeline의 객체 이름 뒤에 '__파라미터이름'을 붙여줘서 인식하도록 해야함!
params = {'tf_idf__ngram_range':[(1,1),(2,2),(3,3)],
         'tf_idf__max_df':[100,200,300],
         'lr_clf__C':[0.1,5,10]}

grid_cv_pipe = GridSearchCV(pipeline, param_grid=params,
                           cv=3, scoring='accuracy',verbose=1)
# Pipeline때문에 벡터화시키기 전의 원본 데이터를 넣어줌
grid_cv_pipe.fit(X_train y_train)
print("최적의 파라미터 값", grid_cv_pipe.best_params_)

y_pred = grid_cv_pipe.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"파이프라인+그리드서치 적용 후 test데이터 정확도:{acc :.4f}")

- 텍스트 분류기 주로 사용되는 모델 - **``로지스틱 리그레션, 나이브베이즈, SVM``**